In [1]:
import pandas as pd
import pylab as pl
import numpy as np
import scipy.optimize as opt
from sklearn import preprocessing
%matplotlib inline 
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
ipl  = pd.read_csv('matches.csv')

In [3]:
ipl.shape

(749, 18)

In [4]:
ipl

,id,season,city,date,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match,venue,umpire1,umpire2,umpire3
0,1,2017,Hyderabad,2017-04-05,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,field,normal,0,Sunrisers Hyderabad,35,0,Yuvraj Singh,"Rajiv Gandhi International Stadium, Uppal",AY Dandekar,NJ Llong,NaN
1,2,2017,Pune,2017-04-06,Mumbai Indians,Rising Pune Supergiant,Rising Pune Supergiant,field,normal,0,Rising Pune Supergiant,0,7,SPD Smith,Maharashtra Cricket Association Stadium,A Nand Kishore,S Ravi,NaN
2,3,2017,Rajkot,2017-04-07,Gujarat Lions,Kolkata Knight Riders,Kolkata Knight Riders,field,normal,0,Kolkata Knight Riders,0,10,CA Lynn,Saurashtra Cricket Association Stadium,Nitin Menon,CK Nandan,NaN
3,4,2017,Indore,2017-04-08,Rising Pune Supergiant,Kings XI Punjab,Kings XI Punjab,field,normal,0,Kings XI Punjab,0,6,GJ Maxwell,Holkar Cricket Stadium,AK Chaudhary,C Shamshuddin,NaN
4,5,2017,Bangalore,2017-04-08,Royal Challengers Bangalore,Delhi Daredevils,Royal Challengers Bangalore,bat,normal,0,Royal Challengers Bangalore,15,0,KM Jadhav,M Chinnaswamy Stadium,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
744,11347,2019,Mumbai,05/05/19,Kolkata Knight Riders,Mumbai Indians,Mumbai Indians,field,normal,0,Mumbai Indians,0,9,HH Pandya,Wankhede Stadium,Nanda Kishore,O Nandan,S Ravi
745,11412,2019,Chennai,07/05/19,Chennai Super Kings,Mumbai Indians,Chennai Super Kings,bat,normal,0,Mumbai Indians,0,6,AS Yadav,M. A. Chidambaram Stadium,Nigel Llong,Nitin Menon,Ian Gould
746,11413,2019,Visakhapatnam,08/05/19,Sunrisers Hyderabad,Delhi Capitals,Delhi Capitals,field,normal,0,Delhi Capitals,0,2,RR Pant,ACA-VDCA Stadium,NaN,NaN,NaN
747,11414,2019,Visakhapatnam,10/05/19,Delhi Capitals,Chennai Super Kings,Chennai Super Kings,field,normal,0,Chennai Super Kings,0,6,F du Plessis,ACA-VDCA Stadium,Sundaram Ravi,Bruce Oxenford,Chettithody Shamshuddin


In [5]:
ipl.describe()

,id,season,dl_applied,win_by_runs,win_by_wickets
count,749.000000,749.000000,749.000000,749.000000,749.000000
mean,1790.005340,2013.435247,0.025367,13.268358,3.381842
std,3461.063331,3.372211,0.157343,23.425815,3.388166
min,1.000000,2008.000000,0.000000,0.000000,0.000000
25%,188.000000,2011.000000,0.000000,0.000000,0.000000
50%,377.000000,2013.000000,0.000000,0.000000,4.000000
75%,566.000000,2016.000000,0.000000,19.000000,6.000000
max,11415.000000,2019.000000,1.000000,146.000000,10.000000


In [6]:
ipl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 749 entries, 0 to 748
Data columns (total 18 columns):
id                 749 non-null int64
season             749 non-null int64
city               749 non-null object
date               749 non-null object
team1              749 non-null object
team2              749 non-null object
toss_winner        749 non-null object
toss_decision      749 non-null object
result             749 non-null object
dl_applied         749 non-null int64
winner             749 non-null object
win_by_runs        749 non-null int64
win_by_wickets     749 non-null int64
player_of_match    749 non-null object
venue              749 non-null object
umpire1            747 non-null object
umpire2            747 non-null object
umpire3            118 non-null object
dtypes: int64(5), object(13)
memory usage: 105.5+ KB


In [7]:
#removing unnecesary columns
X = ipl.drop(['umpire3','id','result', 'date','player_of_match','winner','umpire2','umpire1'], axis=1)

In [8]:
#filling the null values
X = X.fillna(method='ffill')

In [9]:
Y = ipl['winner']

In [10]:
ipl1 = ipl[ipl.isnull().any(axis=1)]
ipl1

,id,season,city,date,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match,venue,umpire1,umpire2,umpire3
0,1,2017,Hyderabad,2017-04-05,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,field,normal,0,Sunrisers Hyderabad,35,0,Yuvraj Singh,"Rajiv Gandhi International Stadium, Uppal",AY Dandekar,NJ Llong,NaN
1,2,2017,Pune,2017-04-06,Mumbai Indians,Rising Pune Supergiant,Rising Pune Supergiant,field,normal,0,Rising Pune Supergiant,0,7,SPD Smith,Maharashtra Cricket Association Stadium,A Nand Kishore,S Ravi,NaN
2,3,2017,Rajkot,2017-04-07,Gujarat Lions,Kolkata Knight Riders,Kolkata Knight Riders,field,normal,0,Kolkata Knight Riders,0,10,CA Lynn,Saurashtra Cricket Association Stadium,Nitin Menon,CK Nandan,NaN
3,4,2017,Indore,2017-04-08,Rising Pune Supergiant,Kings XI Punjab,Kings XI Punjab,field,normal,0,Kings XI Punjab,0,6,GJ Maxwell,Holkar Cricket Stadium,AK Chaudhary,C Shamshuddin,NaN
4,5,2017,Bangalore,2017-04-08,Royal Challengers Bangalore,Delhi Daredevils,Royal Challengers Bangalore,bat,normal,0,Royal Challengers Bangalore,15,0,KM Jadhav,M Chinnaswamy Stadium,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
626,633,2016,Bangalore,2016-05-24,Gujarat Lions,Royal Challengers Bangalore,Royal Challengers Bangalore,field,normal,0,Royal Challengers Bangalore,0,4,AB de Villiers,M Chinnaswamy Stadium,AK Chaudhary,HDPK Dharmasena,NaN
627,634,2016,Delhi,2016-05-25,Sunrisers Hyderabad,Kolkata Knight Riders,Kolkata Knight Riders,field,normal,0,Sunrisers Hyderabad,22,0,MC Henriques,Feroz Shah Kotla,M Erasmus,C Shamshuddin,NaN
628,635,2016,Delhi,2016-05-27,Gujarat Lions,Sunrisers Hyderabad,Sunrisers Hyderabad,field,normal,0,Sunrisers Hyderabad,0,4,DA Warner,Feroz Shah Kotla,M Erasmus,CK Nandan,NaN
629,636,2016,Bangalore,2016-05-29,Sunrisers Hyderabad,Royal Challengers Bangalore,Sunrisers Hyderabad,bat,normal,0,Sunrisers Hyderabad,8,0,BCJ Cutting,M Chinnaswamy Stadium,HDPK Dharmasena,BNJ Oxenford,NaN


In [11]:
# mapping the nominal variable with numerical Varibale
teams = X.team1.unique()
city = X.city.unique()
toss_decision = X.toss_decision.unique()
venue =  X.venue.unique()

In [12]:

j=0
for i in city:
    X.city[X.city==i]=j
    j+=1

/home/koshal/anaconda3/envs/Koshal/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [13]:
#mapping the teams
j=0
for i in teams:
    X.team1[X.team1 == i] = j
    X.team2[X.team2 == i] = j
    X.toss_winner[X.toss_winner==i]=j
    ipl.winner[ipl.winner==i]=j
    Y=j
    j+=1

/home/koshal/anaconda3/envs/Koshal/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/home/koshal/anaconda3/envs/Koshal/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/home/koshal/anaconda3/envs/Koshal/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [14]:
Y=ipl.winner

In [15]:
#mappping the toss_decision
j=0
for i in toss_decision:
    X.toss_decision[X.toss_decision==i]=j
    j+=1


/home/koshal/anaconda3/envs/Koshal/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [16]:
#mappiing venue
j=0
for i in venue:
    X.venue[X.venue==i]=j
    j+=1


/home/koshal/anaconda3/envs/Koshal/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [17]:
X

,season,city,team1,team2,toss_winner,toss_decision,dl_applied,win_by_runs,win_by_wickets,venue
0,2017,0,0,4,4,0,0,35,0,0
1,2017,1,1,3,3,0,0,0,7,1
2,2017,2,2,5,5,0,0,0,10,2
3,2017,3,3,7,7,0,0,0,6,3
4,2017,4,4,6,4,1,0,15,0,4
...,...,...,...,...,...,...,...,...,...,...
744,2019,5,5,1,1,0,0,0,9,5
745,2019,11,8,1,8,1,0,0,6,35
746,2019,25,0,14,14,0,0,0,2,40
747,2019,25,14,8,8,0,0,0,6,40


In [18]:
y= Y.astype('int')

In [19]:
y

0       0
1       3
2       5
3       7
4       4
       ..
744     1
745     1
746    14
747     8
748     1
Name: winner, Length: 749, dtype: int64

In [20]:
X.shape

(749, 10)

In [66]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 749 entries, 0 to 748
Data columns (total 10 columns):
season            749 non-null int64
city              749 non-null object
team1             749 non-null object
team2             749 non-null object
toss_winner       749 non-null object
toss_decision     749 non-null object
dl_applied        749 non-null int64
win_by_runs       749 non-null int64
win_by_wickets    749 non-null int64
venue             749 non-null object
dtypes: int64(4), object(6)
memory usage: 58.6+ KB


In [21]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=4)
print ('Train set:', X_train.shape,  y_train.shape)
print ('Test set:', X_test.shape,  y_test.shape)

Train set: (599, 10) (599,)
Test set: (150, 10) (150,)


In [22]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix,accuracy_score
LR = LogisticRegression().fit(X_train,y_train)
LR

/home/koshal/anaconda3/envs/Koshal/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [23]:
pred= LR.predict(X_test)

In [24]:
accuracy_score(y_test, pred)

0.20666666666666667

In [25]:
result = confusion_matrix(y_test, pred)
result

array([[ 0,  7,  0,  0,  0,  1,  0,  1,  0,  0,  0,  0,  0,  0,  0],
       [ 0, 10,  0,  0,  0,  1,  0,  1,  1,  0,  0,  0,  0,  0,  0],
       [ 0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0, 10,  0,  0,  0,  5,  0,  5,  2,  0,  0,  0,  0,  0,  0],
       [ 0,  7,  0,  0,  0,  6,  0,  1,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  3,  0,  0,  0,  5,  0,  4,  3,  0,  0,  0,  0,  0,  0],
       [ 0,  2,  0,  0,  0,  7,  0,  5,  2,  0,  0,  0,  0,  0,  0],
       [ 0,  4,  0,  0,  1,  4,  0,  5, 10,  1,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  3,  0,  8,  6,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  1,  0,  1,  4,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  1,  0,  0,  2,  0,  0,  0,  0,  0,  0],
       [ 0,  1,  0,  0,  0,  0,  0,  0,  2,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0

In [26]:
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)# Train the model on training data
rf.fit(X_train,y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=1000, n_jobs=None, oob_score=False,
                      random_state=42, verbose=0, warm_start=False)

In [27]:
pred1 = rf.predict(X_test)

In [28]:
accuracy_score(y_test, pred)

0.20666666666666667

In [65]:
X.to_csv('numbers_match.csv', index=False)
y.to_csv('result_match.csv', index=False)

/home/koshal/anaconda3/envs/Koshal/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  


In [59]:
x_tr = np.asarray(X_train)
y_tr = np.asarray(y_train)
x_tr=x_tr*1.0
y_tr=y_tr*1.0

In [60]:
import tensorflow as tf
import keras

In [61]:
model = tf.keras.models.Sequential([tf.keras.layers.Flatten(), 
                                    tf.keras.layers.Dense(10, activation=tf.nn.relu), 
                                    tf.keras.layers.Dense(10, activation=tf.nn.softmax)])

model.compile(optimizer = 'sgd',
              loss = 'sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_tr,y_tr, epochs=5)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type float).

In [34]:
model = keras.Sequential([keras.layers.Dense(units=1, input_shape=[10])])
model.compile(optimizer='sgd', loss='mean_squared_error')
model.fit(X_train,y_train, epochs=10)


Epoch 1/10
599/599 [==============================] - 0s 411us/step - loss: nan
Epoch 2/10
599/599 [==============================] - 0s 162us/step - loss: nan
Epoch 3/10
599/599 [==============================] - 0s 140us/step - loss: nan
Epoch 4/10
599/599 [==============================] - 0s 106us/step - loss: nan
Epoch 5/10
599/599 [==============================] - 0s 161us/step - loss: nan
Epoch 6/10
599/599 [==============================] - 0s 159us/step - loss: nan
Epoch 7/10
599/599 [==============================] - 0s 125us/step - loss: nan
Epoch 8/10
599/599 [==============================] - 0s 86us/step - loss: nan
Epoch 9/10
599/599 [==============================] - 0s 103us/step - loss: nan
Epoch 10/10
599/599 [==============================] - 0s 94us/step - loss: nan


In [41]:
predict = model.predict(X_test)

ValueError: Please provide as model inputs either a single array or a list of arrays. You passed: inputs=     season city team1 team2 toss_winner toss_decision  dl_applied  \
33     2017    2     2     1           2             1           0   
62     2008    5     1     4           1             1           0   
136    2009   15     8     9           9             0           0   
623    2016    9     1     2           2             0           0   
311    2012   10     9     5           5             0           0   
..      ...  ...   ...   ...         ...           ...         ...   
283    2011    8     7    12           7             1           0   
306    2012    6     5     6           6             0           0   
735    2019    7    14     4          14             1           0   
356    2012    0     7    10          10             0           0   
90     2008   11     8     7           7             0           0   

     win_by_runs  win_by_wickets venue  
33           

In [39]:
predict

array([[nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
      